# Resampling Methods: Bootstrap and Jackknife

In [ ]:
import multiprocessing as mp
mp.set_start_method('forkserver')

In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXP_NUM_THREADS'] = '1'

In [ ]:
import numpy as np
np.random.seed(108774)
import scipy.stats.distributions as dists
import matplotlib.pyplot as plt

import statsmodels.api as sm
import ecoglib.estimation.resampling as resampling

## Simple bootstrapping

Both resampling schemes are *nonparameteric* methods to get statistics of a sampling distribution. This is in contrast to a parametric estimator analysis that pre-supposes the sampling distribution. 

**Parametric analysis**

A simple parametric assumption would be a sample $\{x\}_{i}$ of $N$ independent and identically distributed (iid) variates from a friendly distribution with (*finite!*) mean $\mu$ and variance $\sigma^{2}$. It would follow that the sample mean estimator $\hat{\mu}=\sum_{i}(N^{-1}x_{i})$ is a variate with mean $\mu$ and variance $\sigma^{2}/N$. In other words, the standard error (SE) of the mean (SEM) is $\sigma /\sqrt{N}$ (which is itself has an unbiased estimate using the sample variance estimator $\hat{\sigma}^{2}/(N-1)$).

**Nonparametric (bootstrap) analysis**

[Bootstrap](https://en.wikipedia.org/wiki/Bootstrapping_%28statistics%29) tries to infer the sample distribution by creates many point estimates (usually >>100) from synthetic samples. Specifically, bootstrap drawns $N$ times from the original $N$ points with replacement to emulate repeating an experiment to draw $N$ *new* points. The quantiles of the $P$ pseudo estimates are then used to infer statistics of the actual sample distribution.

In [ ]:
N = 13   # sample size
P = 1000  # resamples
# normal samples mean of 5, stdev 2
sigma_sq = 4
mu = 5
sample_dist = dists.norm(loc=mu, scale=sigma_sq ** 0.5)
x_norm = sample_dist.rvs(size=N)

In [ ]:
bs = resampling.Bootstrap(x_norm, P, n_jobs=1)

In [ ]:
with np.printoptions(precision=2, suppress=True):
    print('Real sample:', np.sort(x_norm))
    for n, samp in enumerate(bs.sample()):
        print('Fake sample {}:'.format(n + 1), np.sort(samp))
        if n == 2:
            break

We can draw all bootstrap samples and look at the apparent distribution of the mean estimator compared to the actual distribution of the mean estimator.

In [ ]:
mean_bs = bs.all_samples(estimator=np.mean)  # pseudo-samples from distribution
sample_mean = np.mean(mean_bs)
mean_dist = dists.norm(mu, (sigma_sq / N) ** 0.5)  # actual distribution
x = np.linspace(3, 7, 100)
px = mean_dist.pdf(x)
actual_CI = mean_dist.ppf([0.025, 0.975])

The bootstrap samples are only as good as the original sample. With a fairly low $N$, we'd expect a poor mean estimator.

In [ ]:
plt.figure()
_ = plt.hist(mean_bs, bins='auto', density=True, histtype='step')
_ = plt.plot(x, px)
_ = plt.axvline(sample_mean, color='r', label='mean est.')
_ = plt.axvline(actual_CI[0], color='gray', ls='--')
_ = plt.axvline(actual_CI[1], color='gray', ls='--', label='Actual CI')
plt.legend()

With the $P$ estimator replicates, we can compare the nonparametric quantiles of the mean to the parametric SEM.

In [ ]:
sample_mean = x_norm.mean()
# shift the actual CI to be centered around the sample mean
shift_CI = actual_CI - 5 + sample_mean
# unbiased sem estimator
parametric_sem = x_norm.std() / np.sqrt(N - 1)
parametric_CI = dists.norm.ppf([0.025, 0.975], loc=sample_mean, scale=parametric_sem)
bootstrap_CI = np.percentile(mean_bs, [2.5, 97.5])

plt.figure()
_ = plt.hist(mean_bs, bins='auto', density=True, histtype='step')
_ = plt.axvline(sample_mean, color='r', label='mean est.')
_ = plt.axvline(shift_CI[0], color='gray', ls='--')
_ = plt.axvline(shift_CI[1], color='gray', ls='--', label='mean est. +/- actual CI')
_ = plt.axvline(parametric_CI[0], color='darkorange', ls='--')
_ = plt.axvline(parametric_CI[1], color='darkorange', ls='--', label='+/- parametric CI')
_ = plt.axvline(bootstrap_CI[0], color='yellowgreen', ls='--')
_ = plt.axvline(bootstrap_CI[1], color='yellowgreen', ls='--', label='+/- bootstrap CI')
plt.legend(loc=(1.1, 0.5))

## Jackknife

The [jackknife](https://en.wikipedia.org/wiki/Jackknife_resampling) predates bootstrap as a resampling method. It uses a leave-one-out resampling scheme, rather than resampling with replacement, which means that it can create only (up to) $N$ replicate estimators. For the sample $X_{i}$ (set notation omitted for simplicity), each jackknifed sample is

$X_{-i}=(x_1,\dots,x_{i-1},x_{i+1},\dots,x_{N})$


The jackknife is used to find the mean and error of not-necessarily-simple estimators $\theta(X)$. However, it is often warned that the jackknife should be avoided for nonlinear, non-smooth estimators, such as median and correlation coefficient. The jackknife estimate of the mean is defined as

$\hat{\theta}_{jn}=\frac{1}{N}\sum_i \theta(X_{-i})$

The jackknife does have the advantage of correcting biased estimators (to the leading $1/N$ order of its Taylor expansion). For an estimator $\theta(X)$ that is biased, we can form jackknife *pseudovalues* from the estimator $\theta_{-i}$ applied to each jackknife sample:

$\tilde{\theta}_{-i}=N\bar{\theta}-(N-1)\theta_{-i}$

(where $\bar{\theta}$ is estimated from the *entire* sample). The average of the pseudovalues, which is $N\bar{\theta}-(N-1)\hat{\theta}_{jn}$, is a bias-corrected version of the estimator $\theta$.

The jackknife standard error is defined two ways. The basic jackknife estimator variance is based on jackknifed estimators $\theta_{-i}$

$SE_{1}=\sqrt{\frac{N-1}{N}\sum_{i}(\theta_{-i}-\hat{\theta}_{jn})^2}$

When using pseudovalues, the jackknife SE is

$SE_{2}=\sqrt{\left(N(N-1)\right)^{-1}\sum_{i}(\tilde{\theta}_{-i}-\langle \tilde{\theta} \rangle)^{2}}$

where $\langle \rangle$ notation denotes the sample mean of pseudovalues.

### Bias example: sample variance
The maximum likelihood variance estimator is $N^{-1}\sum_{i}(x_{i}-\bar{x})^{2}$, which has a famous bias of $-\sigma^{2}/n$.

In [ ]:
def svar(x):
    return np.mean((x - np.mean(x)) ** 2)

N = 15   # sample size
# normal samples mean of 5, stdev 2
sigma_sq = 4
mu = 5
sample_dist = dists.norm(loc=mu, scale=sigma_sq ** 0.5)
x_norm = sample_dist.rvs(size=N)

S2 = svar(x_norm)

jn = resampling.Jackknife(x_norm, n_jobs=1)

The ``Jackknife`` object can be used to estimate the bias of ``svar``:

In [ ]:
jn_bias = jn.bias(svar)
known_bias = -sigma_sq / N
print('Jackknife bias: {} (actual bias {})'.format(jn_bias, known_bias))

The jackknife estimate can be made with or without bias correction:

In [ ]:
print('ML estimator:', S2)
print('JN estimator:', jn.estimate(svar, correct_bias=False))
print('JN bias corrected:', jn.estimate(svar, correct_bias=True))

On the other hand, the variance estimator itself has error, which is fairly high compared to the bias. The error can be computed directly (jackknife variance), or returned at the same time as the mean estimate.

In [ ]:
print('Jackknife SE:', jn.variance(svar) ** 0.5)
print('Jackknife est and SE:', *jn.estimate(svar, se=True, correct_bias=True))

## More common use: optimization confidence intervals

Bootstrap is usually used for complex sampling distributions. For example, we can get a confidence interval for parameters that are derived from optimization problems. We can demonstrate this using ordinary least squares (OLS), which again has an easy to compare parametric analysis. (Again, the bootstrap would normally be used for a more complicated distribution, e.g. for optimizing a nonlinear least squares problem.)

Here we have imagined observations of a dependent variable $y$ that linearly varies with the independent variable $x$ with a factor of 1.7. The observations have independent zero mean error with 30% variance.

In [ ]:
x = np.random.randn(55)
e = np.random.randn(55) * (0.3 ** 0.5)
y = 1.7 * x + e

In [ ]:
plt.figure()
plt.scatter(x, y)
plt.xlabel('IV')
plt.ylabel('DV')

Fit an OLS model (slope & intercept) using [statsmodels](https://www.statsmodels.org/dev/user-guide.html#regression-and-linear-models). The size of the confidence intervals here are based on parametric analysis and, interestingly, *only* depend on the independent variable (and not the observations at all!).

In [ ]:
ols_fm = sm.OLS(y, np.c_[np.ones_like(x), x]).fit()
ols_fm.summary()

As the summary states, the 95% CI for the intercept marks it as not necessarily non-zero. The 95% CI slope is near the true value.

In [ ]:
ols_fm.conf_int()

In [ ]:
ols_fm.scale, ols_fm.ssr / ols_fm.df_resid

Based on parametric analysis, the predicted fit has a confidence interval that depends on the estimated error (``ols_fm.scale``) and the "residual degrees of freedom" (N - size-of-model).

In [ ]:
# predict from the model for 100 points
xm = np.linspace(x.min(), x.max(), 100)
pred = ols_fm.get_prediction(np.c_[np.ones_like(xm), xm])
pframe = pred.summary_frame(alpha=0.05)
ym = pframe.mean
ym_lo = pframe.mean_ci_lower
ym_hi = pframe.mean_ci_upper

In [ ]:
xm = np.linspace(x.min(), x.max(), 100)
ym = ols_fm.predict(np.c_[np.ones_like(xm), xm])
plt.figure()
plt.scatter(x, y, color='k', label='data')
plt.plot(xm, ym, color='r', label='predicted mean')
plt.plot(xm, np.c_[ym_lo, ym_hi], color='slategray', ls='--', label='95% CI for mean')
plt.xlabel('IV')
plt.ylabel('DV')
_ = plt.legend()

To get nonparametric confidence intervals, we can bootstrap the OLS fit and check the quantiles of the resulting optimizations. First, define an estimator function that returns optimization parameters:

In [ ]:
def compute_ols(x, y, axis=None):
    fm = sm.OLS(y, np.c_[np.ones_like(x), x]).fit()
    return fm.params

Then create a bootstrapper that resamples BOTH $x$ and $y$ with *the same permutations*. (Another note here: the ``Bootstrap`` object pre-computes the sampling permutations so that work on each sample can be mapped out to subprocesses. Use ``n_jobs=None``, or ``n_jobs=X`` to take advantage of multiple parallel jobs.)

In [ ]:
bootstrapper = resampling.Bootstrap([x, y], 1000)
fits = np.array([p for p in bootstrapper.sample(estimator=compute_ols)])

The average fit and the bootstrapped CI are pretty close to the parametric CI.

In [ ]:
p_mn = fits.mean(axis=0)
p_lo, p_hi = np.percentile(fits, [2.5, 97.5], axis=0)
print('Slope BS: {:.3f} ({:.3f} - {:.3f} 95% CI)'.format(p_mn[1], p_lo[1], p_hi[1]))
print('Intercept BS: {:.3f} ({:.3f} - {:.3f} 95% CI)'.format(p_mn[0], p_lo[0], p_hi[0]))

We can also take these bootstrapped fits to make a CI for the predicted trend. Create a curve for each bootstrap fit and then find the quantiles of these vectors.

In [ ]:
predictions = xm * fits[:, 1, None] + fits[:, 0, None]
ym_lo_bs, ym_hi_bs = np.percentile(predictions, [2.5, 97.5], axis=0)
ym_bs = predictions.mean(0)

In [ ]:
xm = np.linspace(x.min(), x.max(), 100)
plt.figure()
plt.scatter(x, y, color='k', label='data')
plt.plot(xm, ym_bs, color='r', label='BS mean')
plt.plot(xm, np.c_[ym_lo_bs, ym_hi_bs], color='slategray', ls='--', label='BS 95% CI')
plt.xlabel('IV')
plt.ylabel('DV')
_ = plt.legend()

## Computing estimates in parallel

Resampling methods can be resource intensive for any but the most trivial estimators, but the repeated function calls can be done independently on multiple processes. **Due to the overhead involved in creating new processes, going parallel would only make sense for costly estimation functions.**

In [ ]:
from importlib import reload
reload(resampling)

In [ ]:
from time import sleep, time


def slow_mean(x):
    mu = x.mean()
    sleep(0.05)
    return mu


# reuse the normal distribution
N = int(5e6)
x_norm = sample_dist.rvs(size=N)

# run in a single process
t1 = time()
mu_est = resampling.Bootstrap.bootstrap_estimate(x_norm, 100, slow_mean, n_jobs=1)
tc = time() - t1
print('mu estimate: {} ({} sec)'.format(mu_est, tc))

# run in 6 processes
t1 = time()
mu_est = resampling.Bootstrap.bootstrap_estimate(x_norm, 100, slow_mean, n_jobs=6)
tc = time() - t1
print('mu estimate: {} ({} sec)'.format(mu_est, tc))

## Getting all samples

In [ ]:
# reuse the normal distribution
N = int(1e6)
x_norm = sample_dist.rvs(size=N)

t1 = time()
bs = resampling.Bootstrap(x_norm, 1000, n_jobs=1)
all_samps1 = bs.all_samples()
tc = time() - t1
print('serial collect: {} sec'.format(tc))

t1 = time()
bs = resampling.Bootstrap(x_norm, 1000, n_jobs=6)
all_samps2 = bs.all_samples()
tc = time() - t1
print('para collect: {} sec'.format(tc))

In [ ]:
# reuse the normal distribution
N = int(1e6)
x_norm = sample_dist.rvs(size=N)

t1 = time()
bs = resampling.Bootstrap(x_norm, 1000, n_jobs=1, alloc_shared=False)
all_samps1 = bs.all_samples()
tc = time() - t1
print('serial collect: {} sec'.format(tc))

t1 = time()
bs = resampling.Bootstrap(x_norm, 1000, n_jobs=6, alloc_shared=False)
all_samps2 = bs.all_samples()
tc = time() - t1
print('para collect: {} sec'.format(tc))

In [ ]:
del all_samps1
del all_samps2

In [ ]:
import gc
while gc.collect():
    pass

In [ ]:
import cProfile

In [ ]:
cProfile.runctx("bs = resampling.Bootstrap(x_norm, 1000, n_jobs=1); all_samps1 = bs.all_samples()", globals(), locals())

In [ ]:
cProfile.runctx("bs = resampling.Bootstrap(x_norm, 1000, n_jobs=6); all_samps1 = bs.all_samples()", globals(), locals())

In [ ]:
cProfile.runctx("bs = resampling.Bootstrap(x_norm, 1000, n_jobs=6, alloc_shared=False); all_samps1 = bs.all_samples()", globals(), locals())